In [37]:
import json
from pprint import pprint
import pandas as pd
with open('train/cooking_data.json') as data_file:
    data_train = pd.read_json(data_file)
print (data_train)

            cuisine     id                                        ingredients
0             greek  10259  [romaine lettuce, black olives, grape tomatoes...
1       southern_us  25693  [plain flour, ground pepper, salt, tomatoes, g...
2          filipino  20130  [eggs, pepper, salt, mayonaise, cooking oil, g...
3            indian  22213                [water, vegetable oil, wheat, salt]
4            indian  13162  [black pepper, shallots, cornflour, cayenne pe...
5          jamaican   6602  [plain flour, sugar, butter, eggs, fresh ginge...
6           spanish  42779  [olive oil, salt, medium shrimp, pepper, garli...
7           italian   3735  [sugar, pistachio nuts, white almond bark, flo...
8           mexican  16903  [olive oil, purple onion, fresh pineapple, por...
9           italian  12734  [chopped tomatoes, fresh basil, garlic, extra-...
10          italian   5875  [pimentos, sweet pepper, dried oregano, olive ...
11          chinese  45887  [low sodium soy sauce, fresh ginger,

In [39]:
from nltk.stem import WordNetLemmatizer
import re
#data_train['process_ingredients'] = [' , '.join(z).strip() for z in data_train['ingredients']]
data_train['process_ingredients'] = [' '.join([WordNetLemmatizer().lemmatize(re.sub('[^A-Za-z]', ' ', line)) for line in lists]).strip() for lists in data_train['ingredients']] 
#data_train = data_train.drop(columns=['process_ingredients'])
#pprint(data_train.head())
pprint(data_train)

            cuisine     id                                        ingredients  \
0             greek  10259  [romaine lettuce, black olives, grape tomatoes...   
1       southern_us  25693  [plain flour, ground pepper, salt, tomatoes, g...   
2          filipino  20130  [eggs, pepper, salt, mayonaise, cooking oil, g...   
3            indian  22213                [water, vegetable oil, wheat, salt]   
4            indian  13162  [black pepper, shallots, cornflour, cayenne pe...   
5          jamaican   6602  [plain flour, sugar, butter, eggs, fresh ginge...   
6           spanish  42779  [olive oil, salt, medium shrimp, pepper, garli...   
7           italian   3735  [sugar, pistachio nuts, white almond bark, flo...   
8           mexican  16903  [olive oil, purple onion, fresh pineapple, por...   
9           italian  12734  [chopped tomatoes, fresh basil, garlic, extra-...   
10          italian   5875  [pimentos, sweet pepper, dried oregano, olive ...   
11          chinese  45887  

In [40]:
data_train = data_train.drop(columns=['ingredients'])
pprint(data_train)

            cuisine     id                                process_ingredients
0             greek  10259  romaine lettuce black olives grape tomatoes ga...
1       southern_us  25693  plain flour ground pepper salt tomato ground b...
2          filipino  20130  egg pepper salt mayonaise cooking oil green ch...
3            indian  22213                     water vegetable oil wheat salt
4            indian  13162  black pepper shallot cornflour cayenne pepper ...
5          jamaican   6602  plain flour sugar butter egg fresh ginger root...
6           spanish  42779  olive oil salt medium shrimp pepper garlic cho...
7           italian   3735  sugar pistachio nuts white almond bark flour v...
8           mexican  16903  olive oil purple onion fresh pineapple pork po...
9           italian  12734  chopped tomatoes fresh basil garlic extra virg...
10          italian   5875  pimento sweet pepper dried oregano olive oil g...
11          chinese  45887  low sodium soy sauce fresh ginger dr

In [42]:
 data_train = data_train.rename(index=str, columns={"process_ingredients": "ingredients"})

In [43]:
pprint (data_train.head())

       cuisine     id                                        ingredients
0        greek  10259  romaine lettuce black olives grape tomatoes ga...
1  southern_us  25693  plain flour ground pepper salt tomato ground b...
2     filipino  20130  egg pepper salt mayonaise cooking oil green ch...
3       indian  22213                     water vegetable oil wheat salt
4       indian  13162  black pepper shallot cornflour cayenne pepper ...


In [45]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#from nltk.stem import WordNetLemmatizer
corpus = []
for i in range(0, 39774):
    item = re.sub('[^a-zA-Z]',' ',data_train['ingredients'][i])
    item = item.lower()
    item = item.split()
    ps = PorterStemmer()
    
    item = [ps.stem(word) for word in item if not word in set(stopwords.words('english'))]
    item = ' '.join(item)
    corpus.append(item)

In [62]:
#Bag of Words Model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()

labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(data_train['cuisine'])

In [66]:
# Splitting the dataset into the Training set and Validation set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Fitting Multinomial Naive Bayes Model to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

#Predicting for validation set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print (cm[0])

[69  0  0  0  5  2  3  1  2  1 12  0  1  3  2  2  1  1  1  1]


In [70]:
import numpy as np
for i in range(0,np.size(cm,0)):
    print (cm[i])

[69  0  0  0  5  2  3  1  2  1 12  0  1  3  2  2  1  1  1  1]
[20 28  2  2  7  7  2  1 20  1 36  0  2  0  3 29  1  5  0  0]
[ 30   4 139   1  11   0   1   1   8   2  43   1   0   6   1   9   7   5
   1   0]
[ 20   3   0  74  92   2   2   2   1   0  44   7 179   3   6   6   2   2
  42  74]
[17  1  2  2 45  1  0  0  0  1 36  2 18  1  3  2  0  0  3  6]
[81 35  9  0  8 80 14  2 28  7 69  1  8  1 49 70  6 49  0  6]
[ 25   2   2   0   2   1  41   2   3   6  22   0   1   3 103  26   2   2
   0   3]
[ 41  15   1   1   7   2   3 206   2   5 174   5   3   0 138  13   1   2
  10   8]
[16  9  1  1  3  1  0  2 33  3 33  0  5  0  2 10  1  0  0  1]
[169  41  80   7  15  69 249   2  31 203 160   6  17  10 177 125  11 159
   9  11]
[15  2  0  0  2  1  0  1  2  0 92  0  0  5  1  0  0  1  0  0]
[  6   1   0   5   6   0   1  11   0   0  24  73 122   2   4   2   2   1
   5  10]
[  0   0   1   3   2   0   0   0   0   0   5   7 120   1   0   0   0   0
   3   3]
[227   5 106   4  54   3  27   5  14   5 276   

In [71]:
accuracy = (cm[0][0] + cm[1][1])/(cm[0][0] + cm[1][1] + cm[0][1] + cm[1][0])
precision = cm[1][1]/(cm[1][1] + cm[0][1])
recall = cm[1][1]/ (cm[1][1] + cm[1][0])
f1_score = (2 * precision * recall) / (precision + recall)

In [72]:
print (accuracy)

0.8290598290598291
